# 客户画像复赛 数据处理---06&07表特征提取

In [1]:
!ls Data_Update/train/

01_arc_s_95598_wkst_train.tsv  06_cont_info.tsv		11_c_meter.tsv
02_s_comm_rec.tsv	       07_c_rca_cons.tsv	12_a_pay_flow.tsv
03_s_info_oversee.tsv	       08_a_rcved_flow.tsv	out
04_c_cons.tsv		       09_arc_a_rcvbl_flow.tsv	train_label.csv
05_c_cons_prc.tsv	       10_c_meter_read.tsv


## 1 导入必要的包和函数

In [1]:
%pdb 1
debug = 1
import pandas as pd
import numpy as np
import pickle
import re
import io 
import matplotlib.pyplot as plt
%matplotlib inline

# 加载预定义好的函数
# 预定义好的函数：
#    - load_data(file_name, header=0): 加载数据
#    - def show_attribute_detail( data=None, attribute_name=None): 显示数据某个属性的详情
%run data_util_func.py
# 选择过滤方案
PLAN_TYPE = 3

Automatic pdb calling has been turned ON


In [34]:
%who_ls

['FeatureEncoder',
 'data01_appno_contenttype_rdd',
 'data_feature_encoder',
 'debug',
 'extend_train_data',
 'get_accept_content_type',
 'get_highest_sensitive_workerid',
 'get_metrics',
 'io',
 'is_highest_sensitive_worker_dict',
 'load_data',
 'model_predict',
 'model_train',
 'np',
 'pd',
 'pickle',
 'plt',
 'print_function',
 're',
 'save_data',
 'seperate_data_to_classifier',
 'show_attribute_detail',
 'show_df_info',
 'test_data01_df',
 'test_data01_merge_label_df',
 'test_data_labels_df',
 'train_accept_content_type_count_df',
 'train_accept_content_type_sensitive_rate_series',
 'train_custno_in_season4_count_dict',
 'train_custno_in_season4_count_rdd',
 'train_custno_search_action_count_dict',
 'train_custno_search_action_count_rdd',
 'train_data01_a_worker_per_user',
 'train_data01_a_worker_per_user_accept_content_type_count_df',
 'train_data01_custno_handle_month_rdd',
 'train_data01_custno_issearchaction_rdd',
 'train_data01_merge_label_df',
 'train_data01_user_worker_count

In [36]:
del test_data_labels_df

##  2 加载数据 

### 3-15 06表 低保户信息表 06_CONT_INFO  部分特征

#### 2-1-1 训练集

##### 2-1-1-1  加载数据 train/06_cont_info.tsv

In [30]:
# 要特别注意 统一设置 converters={0:unicode}，
# 避免后面连接错误
train_data06_df = load_data(file_name = 'Data_Update/train/06_cont_info.tsv',
                                 converters={0:unicode}
# 7386                             )
print(train_data06_df.shape)
# 2455
print '用户数：%d'%len(train_data06_df['CONS_NO'].unique())

(7386, 4)
用户数：2455


##### 增加特征 - 是否连接上表6

In [62]:
temp = train_data01_a_worker_per_user.merge(
    train_data06_df[['CONS_NO','CONT_TYPE']].drop_duplicates(),
    how='left', left_on='CUST_NO', right_on='CONS_NO', 
)
# 全部先设置 为 1
temp['IS_CONNECT_TO_06TABLE'] = 1
temp.loc[temp['CONS_NO'].isnull(),'IS_CONNECT_TO_06TABLE']=0
print sum(temp['CONS_NO'].notnull()),sum(temp['IS_CONNECT_TO_06TABLE']==1)
print temp.head(),temp.shape

2455 2455
             APP_NO                ID  BUSI_TYPE_CODE  URBAN_RURAL_FLAG  \
0  2015101280016722  1000000527203612               3               2.0   
1  2015101280031388  1000000527304861               3               1.0   
2  2015101280031424  1000000527305097               3               2.0   
3  2015101280031680  1000000527300870               3               2.0   
4  2015101280032506  1000000527305664               3               1.0   

      ORG_NO         HANDLE_TIME                ACCEPT_CONTENT HANDLE_OPINION  \
0    3341020  2015/10/12 7:31:50              【银行代扣】客户咨询银行代扣事宜            NaN   
1    3340901  2015/10/12 9:14:03  【咨询总户号】通过地址（户名、表号）查询户号信息，已告知            NaN   
2      33401  2015/10/12 9:14:16             【查询电费】客户查询电费，已告知。            NaN   
3  334020106  2015/10/12 9:11:37        【咨询总户号】通过地址（户名）查询户号信息。            NaN   
4    3340101  2015/10/12 9:14:48  【咨询总户号】通过地址（户名、表号）查询户号信息，已告知            NaN   

   CALLING_NO  ELEC_TYPE          ...           IS_C

In [20]:
train_data01_a_worker_per_user = train_data01_a_worker_per_user.drop('STATUS',axis=1)

In [64]:
train_data01_a_worker_per_user['IS_CONNECT_TO_06TABLE'] = temp['IS_CONNECT_TO_06TABLE']

In [65]:
sum(train_data01_a_worker_per_user['IS_CONNECT_TO_06TABLE']==1)

2455

In [66]:
train_data01_a_worker_per_user['CONT_TYPE'] = temp['CONT_TYPE']

In [23]:
# 暂时不要，因为现在
# train_data01_a_worker_per_user['STATUS'] = temp['STATUS']

In [67]:
temp = train_data06_df.groupby('CONS_NO')['STATUS'].apply(lambda x: len(set(x)))
temp[temp>1]

CONS_NO
6010276428    2
6120032054    2
7910363317    2
8602022550    2
Name: STATUS, dtype: int64

In [21]:
train_data01_a_worker_per_user.head()

,APP_NO,ID,BUSI_TYPE_CODE,URBAN_RURAL_FLAG,ORG_NO,HANDLE_TIME,ACCEPT_CONTENT,HANDLE_OPINION,CALLING_NO,ELEC_TYPE,...,IS_CONNECT_TO_08TABLE,NUM_OF_RCVBL_PENALTY,IS_CONNECT_TO_07TABLE,CONS_STATUS,RCA_FLAG,ORG_NO_7bit,AVERAGE_RCVBL_AMT,IS_EXCEEDING_RCVBL_YM_GE_1MON,IS_CONNECT_TO_06TABLE,CONT_TYPE
0,2015101280016722,1000000527203612,3,2.0,3341020,2015/10/12 7:31:50,【银行代扣】客户咨询银行代扣事宜,NaN,1196922.0,202.0,...,0,NaN,0,NaN,NaN,NaN,NaN,2.0,0,NaN
1,2015101280031388,1000000527304861,3,1.0,3340901,2015/10/12 9:14:03,【咨询总户号】通过地址（户名、表号）查询户号信息，已告知,NaN,1005636.0,NaN,...,0,NaN,0,NaN,NaN,NaN,NaN,2.0,0,NaN
2,2015101280031424,1000000527305097,3,2.0,33401,2015/10/12 9:14:16,【查询电费】客户查询电费，已告知。,NaN,1007513.0,405.0,...,1,0.0,0,NaN,NaN,3340101.0,1652.304167,0.0,0,NaN
3,2015101280031680,1000000527300870,3,2.0,334020106,2015/10/12 9:11:37,【咨询总户号】通过地址（户名）查询户号信息。,NaN,1073266.0,202.0,...,1,0.0,0,NaN,NaN,3340201.0,51.961667,0.0,0,NaN
4,2015101280032506,1000000527305664,3,1.0,3340101,2015/10/12 9:14:48,【咨询总户号】通过地址（户名、表号）查询户号信息，已告知,NaN,1433592.0,NaN,...,1,0.0,0,NaN,NaN,3340150.0,121.936250,0.0,0,NaN


#### 2-1-1 测试集

##### 2-1-1-1  加载数据 test/06_cont_info_test.tsv

In [4]:
# 要特别注意 统一设置 converters={0:unicode}，
# 避免后面连接错误
test_data06_df = load_data(file_name = 'Data_Update/test/06_cont_info_test.tsv',
                                 converters={0:unicode}
                                )
print(test_data06_df.shape)
print '用户数：%d'%len(test_data06_df['CONS_NO'].unique())

(1505, 4)
用户数：1505


##### 增加特征 - 是否连接上表6

In [16]:
test_data01_a_worker_per_user = test_data01_a_worker_per_user.drop('STATUS',axis=1)

In [13]:
temp = test_data01_a_worker_per_user.merge(
    test_data06_df[['CONS_NO','CONT_TYPE']].drop_duplicates(),
    how='left', left_on='CUST_NO', right_on='CONS_NO', 
)
# 全部先设置 为 0
temp['IS_CONNECT_TO_06TABLE'] = 1
temp.loc[temp['CONS_NO'].isnull(),'IS_CONNECT_TO_06TABLE']=0
print sum(temp['CONS_NO'].notnull())
print temp.head(),temp.shape

1505
             APP_NO                ID  BUSI_TYPE_CODE  URBAN_RURAL_FLAG  \
0  2015101280019632  1000000527215943               3               1.0   
1  2015101280019648  1000000527216033              10               2.0   
2  2015101280019680  1000000527216202               3               2.0   
3  2015101280019873  1000000527218491               3               1.0   
4  2015101280020130  1000000527221199               1               1.0   

      ORG_NO         HANDLE_TIME  \
0    3340780  2015/10/12 7:58:04   
1  334066001  2015/10/12 7:58:26   
2    3340690  2015/10/12 7:58:52   
3    3340810  2015/10/12 8:02:53   
4    3340501  2015/10/12 8:07:10   

                                      ACCEPT_CONTENT HANDLE_OPINION  \
0   【客户咨询一户无电】建议先自行联系有资质电工排查是否为内部故障，客户接受并表示有其他情况再致电。            NaN   
1  【退订】#2061584501；【退订】#2061584602；【退订】#406091928...            NaN   
2                                【查询电费】客户查询本月电费，已告知。            NaN   
3                           【咨询总户号】通过地址、户名查

In [17]:
test_data01_a_worker_per_user.head()

,APP_NO,ID,BUSI_TYPE_CODE,URBAN_RURAL_FLAG,ORG_NO,HANDLE_TIME,ACCEPT_CONTENT,HANDLE_OPINION,CALLING_NO,ELEC_TYPE,...,IS_CONNECT_TO_08TABLE,NUM_OF_RCVBL_PENALTY,IS_CONNECT_TO_07TABLE,CONS_STATUS,RCA_FLAG,ORG_NO_7bit,AVERAGE_RCVBL_AMT,IS_EXCEEDING_RCVBL_YM_GE_1MON,IS_CONNECT_TO_06TABLE,CONT_TYPE
0,2015101280019632,1000000527215943,3,1.0,3340780,2015/10/12 7:58:04,【客户咨询一户无电】建议先自行联系有资质电工排查是否为内部故障，客户接受并表示有其他情况再致电。,NaN,1628699.0,202.0,...,1,0.0,0,NaN,NaN,3340780.0,68.573333,0.0,0,NaN
1,2015101280019648,1000000527216033,10,2.0,334066001,2015/10/12 7:58:26,【退订】#2061584501；【退订】#2061584602；【退订】#406091928...,NaN,1260775.0,201.0,...,1,0.0,0,NaN,NaN,3340660.0,34.745833,0.0,0,NaN
2,2015101280019680,1000000527216202,3,2.0,3340690,2015/10/12 7:58:52,【查询电费】客户查询本月电费，已告知。,NaN,1536321.0,201.0,...,1,0.0,0,NaN,NaN,3340690.0,207.431667,0.0,0,NaN
3,2015101280019873,1000000527218491,3,1.0,3340810,2015/10/12 8:02:53,【咨询总户号】通过地址、户名查询户号信息，已告知,NaN,1576698.0,NaN,...,1,0.0,0,NaN,NaN,3340801.0,20.694444,0.0,0,NaN
4,2015101280020130,1000000527221199,1,1.0,3340501,2015/10/12 8:07:10,【设备故障】客户报修此处线路打火，请处理。客户只能提供此地址，烦请电话联系057486581...,NaN,1089990.0,201.0,...,0,NaN,0,NaN,NaN,NaN,NaN,2.0,0,NaN


In [14]:
test_data01_a_worker_per_user['IS_CONNECT_TO_06TABLE'] = temp['IS_CONNECT_TO_06TABLE']

In [15]:
test_data01_a_worker_per_user['CONT_TYPE'] = temp['CONT_TYPE']

In [42]:
test_data01_a_worker_per_user['STATUS'] = temp['STATUS']

### 3-15 07表 费控用户信息表 07_C_RCA_CONS 部分特征

#### 2-1-1 训练集

##### 2-1-1-1  加载数据 train/07_c_rca_cons.tsv

In [18]:
# 要特别注意 统一设置 converters={0:unicode}，
# 避免后面连接错误
train_data07_df = load_data(file_name = 'Data_Update/train/07_c_rca_cons.tsv',
                                 converters={0:unicode}
                                )
print(train_data07_df.shape)

(22192, 4)


In [24]:
temp = train_data01_a_worker_per_user.merge(
    train_data07_df,
    how='left', left_on='CUST_NO', right_on='CONS_NO', 
)
# 全部先设置 为 0
temp['IS_CONNECT_TO_07TABLE'] = 1
temp.loc[temp['CONS_NO'].isnull(),'IS_CONNECT_TO_07TABLE']=0
print sum(temp['CONS_NO'].notnull())
print temp.head()

In [35]:
train_data07_df.head()

,CONS_NO,ORG_NO,RCA_FLAG,CONS_STATUS
0,1316020607,3340101,0,NaN
1,1316023683,3340101,0,NaN
2,1316032110,3340101,0,NaN
3,1316078898,3340101,0,NaN
4,1316090581,3340101,0,NaN


In [37]:
train_data01_a_worker_per_user['IS_CONNECT_TO_07TABLE'] = temp['IS_CONNECT_TO_07TABLE']

In [38]:
train_data01_a_worker_per_user['CONS_STATUS'] = temp['CONS_STATUS']

In [26]:
train_data01_a_worker_per_user['RCA_FLAG'] = temp['RCA_FLAG']

#### 2-1-1 测试集

##### 2-1-1-1  加载数据 test/07_c_rca_cons_test.tsv

In [29]:
# 要特别注意 统一设置 converters={0:unicode}，
# 避免后面连接错误
test_data07_df = load_data(file_name = 'Data_Update/test/07_c_rca_cons_test.tsv',
                                 converters={0:unicode}
                                )
print(test_data07_df.shape)

(10954, 4)


In [43]:
temp = test_data01_a_worker_per_user.merge(
    test_data07_df,
    how='left', left_on='CUST_NO', right_on='CONS_NO', 
)
# 全部先设置 为 0
temp['IS_CONNECT_TO_07TABLE'] = 1
temp.loc[temp['CONS_NO'].isnull(),'IS_CONNECT_TO_07TABLE']=0
print sum(temp['CONS_NO'].notnull())
print temp.head()

In [46]:
test_data01_a_worker_per_user['IS_CONNECT_TO_07TABLE'] = temp['IS_CONNECT_TO_07TABLE']

In [38]:
test_data01_a_worker_per_user['CONS_STATUS'] = temp['CONS_STATUS']

In [44]:
test_data01_a_worker_per_user['RCA_FLAG'] = temp['RCA_FLAG']